## Submission 2 - Gradient Boosting Regression

I think the following code was used in the second submission, but I didn't take the time on making it clear and pretty since it's not in our final submission.

In [ ]:
def read_files(lFilesPred, sFileLabels, sTypeLabel):
    """
    TODO 
    
    Keyword Arguments: 
    - lFilesPred: 
    - sFileLabels: 
    - sTypeLabels: 
    """
    
    # Read file labels (true labels)
    print("sFileLabels : ", sFileLabels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
    
    # Training-testing data
    iNumFiles=len(lFilesPred)
    #mPredictions=[] #np.zeros((1,iNumFiles))
    #mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    vLabels=[] #true label
    vParID=[] # participant ID
    lDicts=[] 


    # We loop over the predictions files from the tsfresh and SVR
    # lDicts is a list of measurement_id and the predictions obtained 
    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
    print('sTypeLabel : ', sTypeLabel)
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
    
    print('dLabels')
    print(dLabels)
    
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            if dLabels[k]!='NA':
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
                vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
                for j in range(1, iNumFiles):
                    fPred=lDicts[j].get(k)
                if fPred:
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
                    
            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    print('mPredictions and vLabels')
    print(mPredictions.shape)
    print(len(vLabels))
    # Random forest training - regression
    
    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())
    return mPredictions, vLabels

In [ ]:
def RandomForest_fusion(lFilesPred, sFileLabels, iEstimators, sTypeLabel, nudge, rLR, iMD,iRS):

    mPredictions, vLabels = read_files(lFilesPred, sFileLabels, sTypeLabel)
    
    clf=GradientBoostingRegressor(n_estimators=iEstimators, learning_rate=rLR, max_depth=iMD,\
                                      random_state=iRS, loss='ls').fit(mPredictions, vLabels)
    vNewPred=clf.predict(mPredictions)
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]
    
    # we will include the testing data here
    return (vNewPred, vParID, vLabels, vRes1, vRes2, clf)
#print(mPredictions)

In [ ]:
def randomForest_predTest(lTestPred, clf, sDirOut,sName):
    
    lDicts=[] #Will contain the two test predictions
    lmesID=[]
    iNumFiles=len(lTestPred)
    vPredIter=np.zeros((1,iNumFiles))

    for sFilePred in lTestPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
      
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            lmesID.append(k)
            #vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            for j in range(1, iNumFiles):
                fPred=lDicts[j].get(k)
            if fPred:
                vPredIter[0,j]=float(np.asarray(fPred))
            else:
                print(['Unkwnown key:' + k])
                vPredIter[0,j]=float(np.asarray(lDicts[0][k]))

            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    
    vTestPred=clf.predict(mPredictions)
    df = pd.DataFrame({'measurement_id': lmesID, sName:vTestPred})
    df.to_csv(sDirOut+'submissionCisPD'+sName+'.csv', index=False)
    return(lmesID, vTestPred)



### Dyskinesia

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_on_off.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sFilePred2='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTypeLabel='on_off'
n_estimators=300
rLR=0.1
iMD=1
iRS=15
nudge=100
sFileLabels=sFilePred1

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1,\
vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel, nudge, rLR, iMD,iRS)

print('Overall MSE Classif. 1 : ', get_final_score(vTrueLabels,np.array(vParID).astype(int), vRes1))
print('Overall MSE Classif. 2 : ', get_final_score(vTrueLabels, vParID, vRes2))
print('Overall MSE Fusion : ', get_final_score(vPredCross, vParIDCross,vTrueCross))




sTestPred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/On_off_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/'
sName='on_off'
#lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
#print(vTestPred)




### Tremor - Submission 2

In [ ]:
# tsfresh predictions file 
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_tremor.csv'

# SVR predictions file 
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold_all/objs_450_kernel_linear_c_0.02_eps_0.1.csv'

# subchallenge 
sTypeLabel='tremor'

# RandomForest hyperparameter 
n_estimators=5

# Labels
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'


lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion(lFilesPred,
                                                                           sFileLabels,
                                                                           n_estimators,
                                                                           sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.tremor.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/Tremor_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='tremor'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



### Dyskinesia - Submission 2

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_dyskinesia.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'
sTypeLabel='dyskinesia'
n_estimators=5
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold/Dyskinesia_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='z_dyskinesia'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



# Other drafts

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_everyone_svr_bpd import *

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"
iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_knn_bpd2 import *

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"
iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
uniq_subjectid = np.unique(vTraiSubjectId)
enc = OneHotEncoder(handle_unknown='ignore').fit(uniq_subjectid.reshape(-1,1))
print(uniq_subjectid.reshape(-1,1).shape)
print(enc)
temp = enc.transform([[1004]]).toarray()
print(temp.shape)
temp2 = np.repeat(temp,61,axis=0)
temp2.shape
#temp3 = np.random.rand(61,50)
#temp4 = np.concatenate((temp3,temp2),axis=1)
#print(temp4.shape)

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_knn_bpd2 import *

# Get Predictions for Per Patient SVR

## Tremor Best Config 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/resiVecPerPatientSVR_Fold"+fold

    iNeighbors=None

    for iComponent in [250]:#[50, 100, 150, 200, 250, 350, 450, 500, 550]:

        for fCValue in [20.0]:#[2e-13, 2e-07, 0.002, 0.2, 20.0]:
            vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponent)

            pca_knn_bpd2(sFileTrai, sFileTest, sOut,
                         iComponent,
                         iNeighbors,
                         sKernel='linear',
                         fCValue=fCValue,
                         fEpsilon='0.1',
                         bLabelNormalization=False,
                         bPatientPredictionsPkl=True)


    

## Dysk Best Config 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/resiVecPerPatientSVR_Fold"+fold

    iNeighbors=None

    for iComponent in [100]:#[50, 100, 150, 200, 250, 350, 450, 500, 550]:

        for fCValue in [20.0]:#[2e-13, 2e-07, 0.002, 0.2, 20.0]:
            vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponent)

            pca_knn_bpd2(sFileTrai, sFileTest, sOut,
                         iComponent,
                         iNeighbors,
                         sKernel='linear',
                         fCValue=fCValue,
                         fEpsilon='0.1',
                         bLabelNormalization=False,
                         bPatientPredictionsPkl=True)


    

### Test with Mean Normalization SVR 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"


    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecSVR_Fold"+fold

    iNeighbors=None

    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    iComponents=500
    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=True)


    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=True)

### Per Patient - With Mean Normalization

In [ ]:
%load_ext autoreload
%autoreload 2

for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"


    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecSVR_Fold"+fold

    iNeighbors=None

    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    iComponents=500
    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=True)


    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=True)
    
# sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/"
# #sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
# bKnn=False
# bSVR=True
# bEveryoneSVR=False
# bPerSubject=False 
# sDatabase=None
# sSubchallenge=None


In [ ]:
from local.get_final_scores_accuracy import get_final_scores_SVR 
from local.get_final_scores_accuracy import get_final_scores_SVR_lowest_mse_for_subjectid

sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=True 
sDatabase='CIS'
sSubchallenge='dysk'

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

# get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

### Test WITHOUT Mean Normalization

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/resiVecSVR_Fold"+fold

    iNeighbors=None

    iComponents=500
    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=False)

    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=False)
    


In [ ]:
sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=True 
sDatabase='CIS'
sSubchallenge='dysk'

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)


#get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

In [ ]:
%load_ext autoreload
%autoreload 2

from local.get_final_scores_accuracy import get_final_scores_SVR 

sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=False 
sDatabase=None
sSubchallenge=None

get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

### SVR testing around

In [ ]:


sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold0/ivector.scp"
iComponents=400

sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/ResiVecSVR_Fold0"

iNeighbors=None

vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01')


In [ ]:

%load_ext autoreload
%autoreload 2
from pca_knn_bpd import *

sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/ivectors_Testing_Fold0/ivector.scp" 

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/ivectors_Testing_Fold0/ivector.scp" 


iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)
print(vTraiPCA.shape)

# /home/mpgill/BeatPD/BeatPD-CLSP-JHU/pca_everyone_svr_bpd.py 

# sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
# sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"


sOut="/home/mpgill/test/"
sKernel='linear' 
fCValue=0.00000000200000 
fEpsilon=0.1
iNeighbors=None

pca_knn_bpd(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01')

In [ ]:
%load_ext autoreload
%autoreload 2

from get_final_scores_accuracy import get_final_scores_SVR 

sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False

get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR)

In [ ]:
%load_ext autoreload
%autoreload 2

from get_final_scores_accuracy import get_final_scores_SVR_lowest_mse_for_subjectid 

#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"
sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False

#get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR)

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR)

In [ ]:
salut = ['100001_test_5163afe8-a6b0-4ea4-b2ba-9b4501dd5912_10040',
 '100002_test_9152519b-4b57-43be-963c-dd7218495001_10040',
 '100003_test_c7312d73-cb34-4025-b8b8-5299b4033e2f_10040',
 '100004_test_cc730391-146b-420f-9255-c3185061f178_10040',
 '100005_test_cc7b822c-e310-46f0-a8ea-98c95fdb67a1_10041',
 '100006_test_5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a_10041',
 '100007_test_19a3e9ea-fce1-40b7-9457-2618970beb7b_10041',
 '100008_test_e2973da8-1250-4a7c-98d5-b165570a8aeb_10041',
 '100009_test_dc90dc36-b4e5-43ec-b3e8-47c39c763c71_10041']

salut2= ['100001_test_5163afe8-a6b0-4ea4-b2ba-9b4501dd5912_hbv0401',
 '100002_test_9152519b-4b57-43be-963c-dd7218495001_hbv0401',
 '100003_test_c7312d73-cb34-4025-b8b8-5299b4033e2f_hbv0401',
 '100004_test_cc730391-146b-420f-9255-c3185061f178_hbv0401',
 '100005_test_cc7b822c-e310-46f0-a8ea-98c95fdb67a1_hbv0411',
 '100006_test_5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a_hbv0411',
 '100007_test_19a3e9ea-fce1-40b7-9457-2618970beb7b_hbv0411',
 '100008_test_e2973da8-1250-4a7c-98d5-b165570a8aeb_hbv0412',
 '100009_test_dc90dc36-b4e5-43ec-b3e8-47c39c763c71_hbv0412']


sPattern=r'(?<=[_])(1[0-9]{3}(?<=\d)(?!=$)|hbv[0-9]{3})'

vTraiSubjectId = [re.findall(sPattern, fileName)[0] for fileName in salut]

# vTraiSubjectId = np.array(([x[-5:-1] for x in np.array(salut)]))
print(vTraiSubjectId)

vTraiSubjectId2 = [re.findall(sPattern, fileName)[0] for fileName in salut2]
print(vTraiSubjectId2)

In [ ]:
# import pandas as pd 
# import re


# def read_log_results_to_excel(folders, fileName):
#     for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
#         print(folder)
#         value = []
#         liVecDim = [350,400]#[350,400,450,500,550, 600, 650, 700]
#         for ivecDim in liVecDim:
#             sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'

#             #config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
# #             config_pattern = re.compile(r"GLOBAL WINNER PARAMETERS(.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)")


#             textfile = open(sFilePath+fileName)
#             filetext = textfile.read()
#             textfile.close()
# #             print(filetext)
#             matches = re.findall(r"GLOBAL WINNER PARAMETERS(?:.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)",
#                                  filetext)
# #             matches = re.findall(r"GLOBAL WINNER PARAMETERS(.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)",
# #                                  filetext)
#             print(matches)
# #             with open(sFilePath+fileName) as f:
# #                 print(f)
# #                 match = config_pattern.search(line)
# #                 if match:
# #                     #print(match.groups()[0])
# #                     value.append(match.groups()[0])
# #                     #yield line.strip()

#         value = pd.DataFrame(value)
#         value = value.T
#         value.columns = liVecDim
#         display(value)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuPerPatientSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

# # folders=['on_off_noinact_auto30_320fl']
# # fileName='globalAccuEveryoneSVR_Test.log'
# # read_log_results_to_excel(folders, fileName)

In [ ]:
# import pandas as pd 
# import re


# def read_log_results_to_excel(folders, fileName):
#     for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
#         print(folder)
#         value = []
#         liVecDim = [350,400,450,500,550, 600, 650, 700]
#         for ivecDim in liVecDim:
#             sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'

#             #config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
#             config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
# #             config_pattern = re.compile(r"GLOBAL WINNER PARAMETERS(?:.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)")
#             with open(sFilePath+fileName) as f:
#                 for line in f:
#                     match = config_pattern.search(line)
#                     if match:
#                         #print(match.groups()[0])
#                         value.append(match.groups()[0])
#                         #yield line.strip()

#         value = pd.DataFrame(value)
#         value = value.T
#         value.columns = liVecDim
#         display(value)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuPerPatientSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuEveryoneSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

In [ ]:
import pandas as pd 
import re


def read_log_results_to_excel(folders, fileName):
    for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
        print(folder)
        value = []
        liVecDim = [350,450,500,550]
        for ivecDim in liVecDim:
            sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'
            print('Opening : ', sFilePath+fileName)
            textfile = open(sFilePath+fileName)
            filetext = textfile.read()
            textfile.close()
            
            
            result = re.findall(r"Test Final score\s[:| ]\s*(\d*.\d*)",filetext)
            print(result[len(result)-1])
            value.append(result[len(result)-1])
            
        value = pd.DataFrame(value)
        value = value.T
        value.columns = liVecDim
        display(value)

folders=['on_off_noinact_auto30']
fileName='globalAccuPerPatientSVR_Test.log'
read_log_results_to_excel(folders, fileName)

folders=['on_off_noinact_auto30_320fl']
fileName='globalAccuEveryoneSVR_Test.log'
read_log_results_to_excel(folders, fileName)

# Remove inactivity with max 

In [ ]:
def remove_inactivity_max(df_train_label):
    last_filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    display(last_filtered_value)
    for idx in df_train_label.index:
        df_allo = []
        df_train_data = pd.read_csv(
            path_train_data + df_train_label["measurement_id"][idx] + ".csv"
        )

        # Get the absolute max value for X, Y, Z
        max_values = df_train_data.iloc[:, -3:].abs().max()

        # Compute what is 5% of that max
        thresold_energy = 5
        df_treshold = (max_values * thresold_energy) / 100

        # display(df_train_data)
        # Candidates are the frames where X, Y, Z are below that treshold (5% of the max)
        #         df_candidates = df_train_data[(df_train_data.X.abs() <= df_treshold.X) &
        #                                      (df_train_data.Y.abs() <= df_treshold.Y) &
        #                                      (df_train_data.Z.abs() <= df_treshold.Z)]
        # display(df_candidates)
        for idx2 in df_train_data.index:
            # print('df_train_data[idx2]')
            # display(df_train_data.iloc[idx2,-3:])
            last_filtered_value = filtered_value
            filtered_value = last_filtered_value + 0.004 * (
                df_train_data.iloc[idx2, -3:] - last_filtered_value
            )
            y = pd.DataFrame(columns=["Timestamp"])
            y = pd.concat(
                [y, pd.DataFrame([df_train_data.iloc[idx2, 0]], columns=["Timestamp"])],
                ignore_index=True,
            )
            #             print('display y :')
            #             display(y)
            #             print('end display y')

            #             print('display filtered value')
            #             display(pd.DataFrame(filtered_value).transpose())
            #             print('end display filtered value')
            df_allo.append(
                pd.concat([y, pd.DataFrame(filtered_value).transpose()], axis=1)
            )
        #             print('display df_allo')
        #             display(df_allo)

        # FIXME : change the name df_allo
        df_allo = pd.DataFrame(df_allo, columns=("Timestamp", "X", "Y", "Z"))

        df_allo.plot(x="Timestamp", legend=True, subplots=True, title="allo")
        stop()


#         v_candidate_x = pd.DataFrame({'Candidate':list(0)})
#         v_candidate_x = np.where(df_train_data.X.abs() <= df_treshold.X, 1, 0)

In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np 
rot_ang = 45

rot = np.random.randint(-rot_ang,rot_ang,size=1)[0]
print('rot : ', rot)
print([rot]*3)
r = R.from_euler('xyz', [rot]*3, degrees=True)
print(r)
rot_mat = r.as_dcm()
print('rot_mat : ', rot_mat)
# temp_train_X = np.dot(temp_train_X, rot_mat)

# Tsfresh experiments

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from lofo import LOFOImportance, Dataset, plot_importance
import numpy as np 
import pandas as pd
all_labels = pd.read_csv('/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv')

all_obj = ["on_off", "tremor", "dyskinesia"]
obj = "tremor"

all_labels = all_labels.drop(list(set(all_obj) - set([obj])), axis=1)

all_features = pd.read_csv('/home/mpgill/BeatPD/BeatPD-CLSP-JHU/tsfresh/submit/features/cis-pd.training.csv')
all_features_labels = pd.merge(all_features, all_labels, on=["measurement_id"])

all_features_labels = all_features_labels.dropna(subset=[obj])

# Compute the average for a speaker of all the features and add those new features to all_features_label
avg = all_features_labels.groupby('subject_id').mean().reset_index().add_prefix('sp_').rename(columns={'sp_subject_id':'subject_id'})
all_features_labels = pd.merge(all_features_labels, avg, on='subject_id')

# Performing global normalization, removing the mean for the features and the labels
remove = []
for i in all_features_labels.columns:
    # if it doesn't start with sp and its sp_ is in  all_features_labels
    if not i.startswith('sp_') and 'sp_' + i in all_features_labels.columns:
        # We remove the sp_ column
        remove.append('sp_' + i)
        # We replace the feature with their value minus the mean of the subject 
        all_features_labels[i] = all_features_labels[i] - all_features_labels['sp_' + i]
# We drop the sp_ columns which were only useful to substract the mean
all_features_labels = all_features_labels.drop(remove, axis=1)

all_features_labels = pd.merge(all_features_labels, pd.read_csv('/home/mpgill/BeatPD/BeatPD-CLSP-JHU/tsfresh/submit/data/order.csv'), how='inner', on=["measurement_id"])
weight = all_features_labels.groupby(['subject_id', 'fold_id']).count().reset_index()[["subject_id", "fold_id", obj]].rename(columns={obj: 'spcount'})
all_features_labels = pd.merge(all_features_labels, weight, on=['subject_id', 'fold_id'])
subject_id = pd.get_dummies(all_features_labels.subject_id, columns='subject_id', prefix='spk_')
all_features_labels = pd.concat([all_features_labels, subject_id], axis=1)

all_features_labels["tremor"] = all_features_labels["tremor"].astype(int)
display(all_features_labels["tremor"])
# define the binary target and the features
dataset = Dataset(df=all_features_labels, target="tremor", features=[col for col in all_features_labels.columns if col != "tremor"])

# Use predefined cv 
foldid = np.array(all_features_labels['fold_id']).astype(int)
from sklearn.model_selection import PredefinedSplit
cv = PredefinedSplit(foldid)

# define the validation scheme and scorer. The default model is LightGBM
lofo_imp = LOFOImportance(dataset, cv=cv, scoring="neg_mean_squared_error")

# get the mean and standard deviation of the importances in pandas format
importance_df = lofo_imp.get_importance()

# plot the means and standard deviations of the importances
plot_importance(importance_df, figsize=(12, 20))

In [ ]:
import pandas as pd
all_labels = pd.read_csv('/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv')

In [ ]:
all_obj = ["on_off", "tremor", "dyskinesia"]
obj = "tremor"

all_labels = all_labels.drop(list(set(all_obj) - set([obj])), axis=1)

In [ ]:
all_features = pd.read_csv('/home/mpgill/BeatPD/BeatPD-CLSP-JHU/tsfresh/submit/features/cis-pd.training.csv')
all_features_labels = pd.merge(all_features, all_labels, on=["measurement_id"])

In [ ]:
all_features_labels = all_features_labels.dropna(subset=[obj])

In [ ]:
# Compute the average for a speaker of all the features and add those new features to all_features_label
avg = all_features_labels.groupby('subject_id').mean().reset_index().add_prefix('sp_').rename(columns={'sp_subject_id':'subject_id'})
all_features_labels = pd.merge(all_features_labels, avg, on='subject_id')


In [ ]:
# Performing global normalization, removing the mean for the features and the labels
remove = []
for i in all_features_labels.columns:
    # if it doesn't start with sp and its sp_ is in  all_features_labels
    if not i.startswith('sp_') and 'sp_' + i in all_features_labels.columns:
        # We remove the sp_ column
        remove.append('sp_' + i)
        # We replace the feature with their value minus the mean of the subject 
        all_features_labels[i] = all_features_labels[i] - all_features_labels['sp_' + i]
# We drop the sp_ columns which were only useful to substract the mean
all_features_labels = all_features_labels.drop(remove, axis=1)

In [ ]:
all_features_labels = pd.merge(all_features_labels, pd.read_csv('/home/mpgill/BeatPD/BeatPD-CLSP-JHU/tsfresh/submit/data/order.csv'), how='inner', on=["measurement_id"])

In [ ]:
weight = all_features_labels.groupby(['subject_id', 'fold_id']).count().reset_index()[["subject_id", "fold_id", obj]].rename(columns={obj: 'spcount'})

In [ ]:
type(all_features_labels)

In [ ]:
all_features_labels = pd.merge(all_features_labels, weight, on=['subject_id', 'fold_id'])

In [ ]:
subject_id = pd.get_dummies(all_features_labels.subject_id, columns='subject_id', prefix='spk_')

In [ ]:
all_features_labels = pd.concat([all_features_labels, subject_id], axis=1)

In [ ]:
all_features_labels.isnull().sum()

In [ ]:
import numpy as np 
Y = np.array(all_features_labels[obj])

In [ ]:
#all_features_labels["tremor"]

In [ ]:
W = np.array(all_features_labels['spcount']) ** -0.5

In [ ]:
#all_features_labels['spcount']

In [ ]:
#all_features_labels.keys().to_list()

In [ ]:
foldid = np.array(all_features_labels['fold_id']).astype(int)
from sklearn.model_selection import PredefinedSplit
cv = PredefinedSplit(foldid)

# X contains all the features 
X = all_features_labels.drop([obj, 'subject_id', 'measurement_id', 'spcount', 'fold_id'], axis=1).astype(pd.np.float32).values
print(X.shape)
print(foldid.shape)

In [ ]:
avg.keys()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
# xgboost
clf = xgb.XGBRegressor()

best_params = {'subsample': 1.0, 'silent': False, 'gamma': 1.0, 'reg_lambda': 100.0, 'min_child_weight': 0.5, 'objective': 'reg:squarederror', 'learning_rate': 0.3, 'max_depth': 2, 'colsample_bytree': 0.8, 'n_estimators': 100, 'colsample_bylevel': 0.5}



In [ ]:

results = []
baselines = []

preds = []

lambda_value = None
random_forest = None
all_spks = all_features_labels['subject_id'].unique()

columns_to_exclude = (all_features_labels.columns[(all_features_labels.dtypes.values != np.dtype('float64'))]).to_list()
columns_to_exclude.remove('fold_id')

for i in range(5):
    ##test = pd.read_csv(sys.argv[i]).squeeze()
    ##idx = all_features_labels['measurement_id'].isin(test)

    # Filter all_features_labels to only the measurements of the current fold
    idx = all_features_labels['fold_id'] == i

    ##tr_w = all_features_labels[~idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##tr = pd.merge(all_features_labels[~idx], tr_w, on='subject_id')
    
    # Drop the measurements that are not from the current fold 
    tr = all_features_labels[~idx].drop(['fold_id'], axis=1)

    # Data augmentation with a lambda
    if lambda_value is not None:
        for spk in all_spks: 
            # Filter training data for that speaker 
            tr_subject = tr.loc[tr['subject_id'] == spk]

            # FIXME: The mean number of recordings changes per fold so to make it easier 
            # I'm just gonna use the mean for the data augmented values
            mean_spcount = int(tr_subject['spcount'].mean())

            # Apply the lambda on the training features except the columns that are not float
            modDfObj1 = tr_subject[tr_subject.columns.difference(columns_to_exclude)].apply(lambda x: x * lambda_value, axis=1, result_type='broadcast')
            modDfObj2 = tr_subject[tr_subject.columns.difference(columns_to_exclude)].apply(lambda x: x * (1-lambda_value), axis=1, result_type='broadcast')
            
            df_data_aug = []
            # First loop to go over the rows
            for index, measurement1 in modDfObj1.iterrows():
                # Second rows to go over the loop except the same two rows
                for index2, measurement2 in modDfObj2.iterrows():
                    if index >= index2:
                        continue
                    df_data_aug.append(measurement1.add(measurement2).to_list())
            df_data_aug = pd.DataFrame(df_data_aug, columns=modDfObj1.columns)
            
            # Add again the columns we just removed 
            #modDfObj1 = pd.concat([modDfObj1, tr_subject[columns_to_exclude]], axis=1)
            subjects_columns_title = [x for x in columns_to_exclude if x.startswith("spk_")]
            # Select the first row of spk_ elements 
            subject_row = tr_subject.iloc[0,tr_subject.columns.str.startswith("spk_")]
            subject_row = subject_row.append(pd.Series(mean_spcount, index=["spcount"]))
            # Duplicate that first row for the number of elements we need
            # full_subjects = pd.concat([subject_row]*len(df_data_aug), ignore_index=True)
            #FIXME NEEDED?
            # full_subjects = pd.DataFrame(full_subjects, columns=subjects_columns_title)

            full_subjects = pd.concat([pd.DataFrame(subject_row).T]*len(df_data_aug), ignore_index=True)
            df_data_aug = pd.concat([df_data_aug, full_subjects], axis=1)
            #df_data_aug = pd.concat([df_data_aug, full_subjects], axis=1)

            # Append the augmented dataframe to the original dataframe
            tr = pd.concat([tr, df_data_aug], ignore_index=True)
            print('After spk ', str(spk), ' tr shape is : ', tr.shape)
#         tr.to_csv("mdl/tr_cis-pd_{0}_fold_{1}_lamb_{2}.csv".format(obj, i, lambda_value), index=False)
    # else:
        # print(tr.keys())
        # If we are using lambda data augmentation, these columns were already removed 
        # tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)
    print('obj is : ', obj)
    train_y = tr[obj].astype(pd.np.float32) # training labels 
    train_weight = tr['spcount'] ** -0.5 # training weight
    tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)


    ##te_w = all_features_labels[idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##te = pd.merge(all_features_labels[idx], te_w, on='subject_id')
    
    # Drop the measurements that are used in the training of this fold, so we keep [idx] instead of [~idx]
    te = all_features_labels[idx].drop(['fold_id'], axis=1)
    # print('te !!! : ', te)

#     print('te[spcount] : ', te['spcount'])
    test_weight = te['spcount'] ** -0.5 # test weight 
#     print('test_weigth : ', test_weight)
    test_y = te[obj].astype(pd.np.float32) # testing labels
    sub = te['subject_id']
    sid = te.subject_id
    test_measurement_id = te.measurement_id
    te = te.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

    if random_forest:
        # Random Forest Regressor 
        print('Using Random Forest Regressor')
        clf = RandomForestRegressor(**best_params)
        clf.fit(tr, train_y, sample_weight=train_weight)
    else:
        # XGBoost 
        print('Using XGBOOST')
        clf = xgb.XGBRegressor(**best_params)
        #clf = xgb.XGBClassifier(**params)

        # Fit for the xgboost 
        clf.fit(
            tr, train_y,
            sample_weight=train_weight,
            eval_set=[(tr, train_y), (te, test_y)],
            #eval_metric=',
            sample_weight_eval_set=[train_weight, test_weight],
            verbose=0,
            early_stopping_rounds=100
        )
#     print(' AVG !!! ')
#     print(avg.keys())
#     print(avg.sp_tremor)
#     print('end avg')
    pred = clf.predict(te).clip(0, 4)
    # print('pred : ', pred)
    mse = (pred - test_y) ** 2
    #mse = test_y.to_numpy() ** 2
    mse2 = test_y ** 2
#     display(te)
    ret = pd.concat([sub, mse], axis=1)
#     display(ret)
    #ret.to_csv('tmp.csv')
    # Mean MSE Per subject ID 
#     print('ret.groupby(subject_id) : ', ret.groupby('subject_id'))
#     display(ret.groupby('subject_id').sum()[obj])
#     print(ret)
    mse_marie = (ret.groupby('subject_id').sum())[obj].to_numpy()
    
    # Number of files per subject_id
    cnt_marie = (ret.groupby('subject_id').count())[obj].to_numpy()
    print('mse_marie : ', mse_marie)
    print('cnt_marie : ', cnt_marie)
#     cnt = cnt ** 0.5
    msek_per_fold = (1/cnt_marie) * mse_marie
    print('msek_per_fold : ', msek_per_fold)
    
    res = pd.DataFrame(data={'measurement_id': test_measurement_id, 'subject_id': sid, obj: pred})

    # We merge the preds with the avg on subject_id, the goal is to keep as label the standardized one
    res = pd.merge(res, avg, on='subject_id')
    
    # Add the sp_obj value to tremor 
    res[obj] += res['sp_' + obj]
    # Res now only contains the measurement_id and the standardized label
    res = res[["measurement_id", obj]]

    preds.append(res)

    print('mse : ', mse)
#     print('test_weigth : ', test_weight)
#     print('before squeeze : ', ((mse * test_weight).sum() / test_weight.sum()))
#     print('after squeeze : ', ((mse * test_weight).sum() / test_weight.sum()).squeeze())
    results.append(((mse * test_weight).sum() / test_weight.sum()).squeeze())
    baselines.append(((mse2 * test_weight).sum() / test_weight.sum()).squeeze())
preds = pd.concat(preds)
print('preds ', preds)
    
print('results : ', results)
#     raise KeyboardInterrupt

In [ ]:
# print(preds)
df_preds = pd.concat(preds)
display(df_preds)

In [ ]:
all_labels.keys()

In [ ]:
true_labels = all_labels[["measurement_id", obj, 'subject_id']]
true_labels = true_labels.dropna(subset=[obj])
true_labels = true_labels.rename(columns={obj: 'true_'+obj})
preds_true_labels = pd.merge(df_preds, true_labels, on='measurement_id')
# print(all_labels[['subject_id']])
# preds_true_labels = pd.merge(preds_true_labels, all_labels[['subject_id']], on='measurement_id')
display(preds_true_labels)

In [ ]:
all_spks = preds_true_labels['subject_id'].unique()
print(all_spks)
msek_subject = []
for spk in all_spks: 
    spk_preds_true_label = preds_true_labels[preds_true_labels['subject_id'] == spk]
    msek = ((spk_preds_true_label['true_'+obj] - spk_preds_true_label[obj]) ** 2).sum() * (1/len(spk_preds_true_label))
    print(spk , ' : ', msek)
    msek_subject.append(msek)

print('msek mean')
print(sum(msek_subject) / 13)
display((pd.DataFrame(msek_subject)))

In [ ]:
msek_subject

In [ ]:
(true_labels.groupby('subject_id').count())

# Old experiments 

In [ ]:
clf = xgb.XGBRegressor(**best_params)

In [ ]:
print(tr.isnull().sum())
clf.fit(
        tr, train_y,
        sample_weight=train_weight,
        eval_set=[(tr, train_y), (te, test_y)],
        #eval_metric=',
        sample_weight_eval_set=[train_weight, test_weight],
        verbose=0,
        early_stopping_rounds=100
    )

In [ ]:
print(tr.describe())
print(test_y.describe())
tr.isnull().sum()

In [ ]:
pred = clf.predict(te).clip(0, 4)
pred

In [ ]:
lambda_value = 1

all_spks = all_features_labels['subject_id'].unique()

columns_to_exclude = (all_features_labels.columns[(all_features_labels.dtypes.values != np.dtype('float64'))]).to_list()
columns_to_exclude.remove('fold_id')

for i in range(5):
    ##test = pd.read_csv(sys.argv[i]).squeeze()
    ##idx = all_features_labels['measurement_id'].isin(test)

    # Filter all_features_labels to only the measurements of the current fold
    idx = all_features_labels['fold_id'] == i

    ##tr_w = all_features_labels[~idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##tr = pd.merge(all_features_labels[~idx], tr_w, on='subject_id')
    
    # Drop the measurements that are not from the current fold
    tr = all_features_labels[~idx].drop(['fold_id'], axis=1)
    
        # Data augmentation with a lambda
    if 5 == 6:
        print('Loading model : ', "mdl/tr_cis-pd_{0}_fold_{1}_lamb_{2}.csv".format(obj, i, lambda_value))
        if sys.argv[5] == "load_mdl":
            tr = pd.read_csv("mdl/tr_cis-pd_{0}_fold_{1}_lamb_{2}.csv".format(obj, i, lambda_value), index_col=[0])
    elif lambda_value is not None:
        for spk in all_spks: 
            # Filter training data for that speaker 
            tr_subject = tr.loc[tr['subject_id'] == spk]
            
            # FIXME: The mean number of recordings changes per fold so to make it easier 
            # I'm just gonna use the mean for the data augmented values
            mean_spcount = int(tr_subject['spcount'].mean())
            
            
            # Apply the lambda on the training features except the columns that are not float
            modDfObj1 = tr_subject[tr_subject.columns.difference(columns_to_exclude)].apply(lambda x: x * lambda_value, axis=1, result_type='broadcast')
            modDfObj2 = tr_subject[tr_subject.columns.difference(columns_to_exclude)].apply(lambda x: x * (1-lambda_value), axis=1, result_type='broadcast')
            
            df_data_aug = []
            # First loop to go over the rows
            for index, measurement1 in modDfObj1.iterrows():
                # Second rows to go over the loop except the same two rows
                for index2, measurement2 in modDfObj2.iterrows():
                    if index >= index2:
                        continue
                    df_data_aug.append(measurement1.add(measurement2).to_list())
            df_data_aug = pd.DataFrame(df_data_aug, columns=modDfObj1.columns)
            
            # Add again the columns we just removed 
            #modDfObj1 = pd.concat([modDfObj1, tr_subject[columns_to_exclude]], axis=1)
            subjects_columns_title = [x for x in columns_to_exclude if x.startswith("spk_")]
            # Select the first row of spk_ elements 
            subject_row = tr_subject.iloc[0,tr_subject.columns.str.startswith("spk_")]
            subject_row = subject_row.append(pd.Series(mean_spcount, index=["spcount"]))
            
            # Duplicate that first row for the number of elements we need
            # full_subjects = pd.concat([subject_row]*len(df_data_aug), ignore_index=True)
            #FIXME NEEDED?
            # full_subjects = pd.DataFrame(full_subjects, columns=subjects_columns_title)

            full_subjects = pd.concat([pd.DataFrame(subject_row).T]*len(df_data_aug), ignore_index=True)
            
            df_data_aug = pd.concat([df_data_aug, full_subjects], axis=1)
            #df_data_aug = pd.concat([df_data_aug, full_subjects], axis=1)
            
            # Append the augmented dataframe to the original dataframe
            tr = pd.concat([tr, df_data_aug], ignore_index=True)
            
            print('After spk ', str(spk), ' tr shape is : ', tr.shape)
#             print(tr.isnull().sum())
#             raise KeyboardInterrupt
#         tr.to_csv("mdl/tr_cis-pd_{0}_fold_{1}_lamb_{2}.csv".format(obj, i, lambda_value), index=False)
    else:
        # If we are using lambda data augmentation, these columns were already removed 
        tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

    train_y = tr[obj].astype(pd.np.float32) # training labels 
    train_weight = tr['spcount'] ** -0.5 # training weight
    tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)
    print('train_weight : ', train_weight)

    ##te_w = all_features_labels[idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##te = pd.merge(all_features_labels[idx], te_w, on='subject_id')
    
    # Drop the measurements that are used in the training of this fold, so we keep [idx] instead of [~idx]
    te = all_features_labels[idx].drop(['fold_id'], axis=1)
    print('te !!! : ', te)
    test_weight = te['spcount'] ** -0.5 # test weight 
    test_y = te[obj].astype(pd.np.float32) # testing labels
    #sub = te['subject_id']
    sid = te.subject_id
    test_measurement_id = te.measurement_id
    te = te.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

    # XGBoost 
    clf = xgb.XGBRegressor(**best_params)
    #clf = xgb.XGBClassifier(**params)

    # Random Forest Regressor 
    #clf = RandomForestRegressor(**best_params)
    #clf.fit(tr, train_y, sample_weight=train_weight)

    # Fit for the xgboost 
    clf.fit(
        tr, train_y,
        sample_weight=train_weight,
        eval_set=[(tr, train_y), (te, test_y)],
        #eval_metric=',
        sample_weight_eval_set=[train_weight, test_weight],
        verbose=0,
        early_stopping_rounds=100
    )
    pred = clf.predict(te).clip(0, 4)
    print('pred : ', pred)
            

In [ ]:
subject_row.append(pd.Series(mean_spcount, index=["spcount"]))

In [ ]:
train_y.isnull().values.any()
# tr.isnull().sum().to_string()
# tr.isnull().sum().sum()
print(te.isnull().T.any().T.sum())

In [ ]:
tr['spcount']

In [ ]:
print(train_weight.isnull().sum())

In [ ]:
print(clf)
for item in zip(feature_list_transform, xgb.feature_importances_):
    print("{1:10.4f} - {0}".format(item[0],item[1]))

In [ ]:
booster = model.get_booster()
dump = booster.get_dump()[0]
assert not ('\nnan\n' in dump or '\n-nan\n' in dump or '\ninf\n' in dump or '\n-inf\n' in dump), "Got non-finite weights from fit: %s" % str(dump)

In [ ]:
tr_subject.iloc[0,tr_subject.columns.str.startswith("spk_")]

In [ ]:
# def data_aug_lambda(measurement1, measurement2, lambda_value):
#     modDfObj1 = measurement1.apply(lambda x: x * lambda_value, axis=1)
#     modDfObj2 = measurement2.apply(lambda x: x * (1-lambda_value), axis=1)
#     mea_new = modDfObj1.add(modDfObj2)
#     return mea_new

In [ ]:
def data_aug_lambda(modDfObj1, modDfObj2):
    df_data_aug = []

    # First loop to go over the rows
    for index, measurement1 in modDfObj1.iterrows():
        # Second rows to go over the loop except the same two rows
        for index2, measurement2 in modDfObj2.iterrows():
            if index >= index2:
                continue
            df_data_aug.append(measurement1.add(measurement2).to_list())
    df_data_aug = pd.DataFrame(df_data_aug, columns=modDfObj1.columns)

    
    return df_data_aug

In [ ]:
columns_to_exclude

## Old drafts

In [ ]:
salut1 = pd.DataFrame([['a', 6, 4, 10, 2], ['a', 4, 10, 12, 4]], columns=['subject_id','X','Y','Z','label'])
# salut2 = pd.DataFrame([[6, 4, 10, 2], [4, 10, 12, 4], [10, 8, 16, 22], [2, 10, 16, 20]])
lambda_value = 0.5

modDfObj1 = salut1[salut1.columns.difference(['subject_id'])].apply(lambda x: x * lambda_value, axis=1, result_type='broadcast')
modDfObj2 = salut1[salut1.columns.difference(['subject_id'])].apply(lambda x: x * (1-lambda_value), axis=1, result_type='broadcast')
display(salut1)
display(salut2)
display('modDfObj1 : ', modDfObj1)
display('modDfObj2 : ', modDfObj2)
df_data_aug2 = data_aug_lambda(modDfObj1, modDfObj2)
display(df_data_aug2)
# df_data_aug2 = pd.concat([df_data_aug2, salut1['subject']], axis=1)
subjects_columns_title = [x for x in columns_to_exclude if x.startswith("spk_")]

# Select the first row of spk_ elements 
subjects = salut1.loc[0,salut1.columns.str.startswith("subject_id")]

# Duplicate that first row for the number of elements we need
full_subjects = pd.concat([subjects]*len(df_data_aug2), ignore_index=True)
full_subjects = pd.DataFrame(full_subjects)
full_subjects.columns = ['subject_id']
# Assign the column name 
display(full_subjects)
print(df_data_aug2.keys())
# Merge the data augmented with the spk information
df_data_aug2 = pd.concat([df_data_aug2, full_subjects], axis=1)
print(df_data_aug2.keys())
print(df_data_aug2)
result = pd.concat([salut1, df_data_aug2], ignore_index=True)
display(result)

In [ ]:
salut1 = pd.DataFrame([['a', 6, 4, 10, 2], ['b', 4, 10, 12, 4], ['c', 10, 8, 16, 22], ['d',2, 10, 16, 20]], columns=['subject','1','2','3','4'])
# salut2 = pd.DataFrame([[6, 4, 10, 2], [4, 10, 12, 4], [10, 8, 16, 22], [2, 10, 16, 20]])
lambda_value = 0.5

modDfObj1 = salut1[salut1.columns.difference(['subject'])].apply(lambda x: x * lambda_value, axis=1, result_type='broadcast')
modDfObj2 = salut1[salut1.columns.difference(['subject'])].apply(lambda x: x * (1-lambda_value), axis=1, result_type='broadcast')
display(salut1)
display(salut2)
display('modDfObj1 : ', modDfObj1)
display('modDfObj2 : ', modDfObj2)
df_data_aug2 = data_aug_lambda(modDfObj1, modDfObj2)
display(df_data_aug2)
# df_data_aug2 = pd.concat([df_data_aug2, salut1['subject']], axis=1)
subjects_columns_title = [x for x in columns_to_exclude if x.startswith("spk_")]

# Select the first row of spk_ elements 
subjects = salut1.loc[0,salut1.columns.str.startswith("subject")]

# Duplicate that first row for the number of elements we need
full_subjects = pd.concat([subjects]*len(df_data_aug2), ignore_index=True)
full_subjects = pd.DataFrame(full_subjects)
# full_subjects.columns = subjects_columns_title
# Assign the column name 
display(full_subjects)
print(df_data_aug2.keys())
# Merge the data augmented with the spk information
df_data_aug2 = pd.concat([df_data_aug2, full_subjects], axis=1)
print(df_data_aug2.keys())
print(df_data_aug2)
result = pd.concat([modDfObj1, df_data_aug2], ignore_index=True)
display(result)

In [ ]:
lambda_value = 0.3 

for i in range(5):
    ##test = pd.read_csv(sys.argv[i]).squeeze()
    ##idx = all_features_labels['measurement_id'].isin(test)

    # Filter all_features_labels to only the measurements of the current fold
    idx = all_features_labels['fold_id'] == i

    ##tr_w = all_features_labels[~idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##tr = pd.merge(all_features_labels[~idx], tr_w, on='subject_id')
    
    # Drop the measurements that are not from the current fold 
    tr = all_features_labels[~idx].drop(['fold_id'], axis=1)
    train_weight = tr['spcount'] ** -0.5 # training weight 
    print(tr)
    # Data augmentation with a lambda 
    if lambda_value is not None: 
        df2 = df[tr.columns.difference(['subject_id', 'spcount'])]
        df2 = df2.mul(lambda_value)
        tr.append(tr.mul(lambda_value))
        
    train_y = tr[obj].astype(pd.np.float32) # training labels 
    tr = tr.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

    ##te_w = all_features_labels[idx].groupby('subject_id').count().reset_index()[["subject_id", obj]].rename(columns={obj: 'spcount'})
    ##te = pd.merge(all_features_labels[idx], te_w, on='subject_id')
    
    # Drop the measurements that are used in the training of this fold, so we keep [idx] instead of [~idx]
    te = all_features_labels[idx].drop(['fold_id'], axis=1)
    test_weight = te['spcount'] ** -0.5 # test weight 
    test_y = te[obj].astype(pd.np.float32) # testing labels
    #sub = te['subject_id']
    sid = te.subject_id
    test_measurement_id = te.measurement_id
    te = te.drop([obj, 'subject_id', 'measurement_id', 'spcount'], axis=1).astype(pd.np.float32)

In [ ]:
numerics = ['int16', 'int32', 'int64']

newdf = tr.select_dtypes(include=numerics)
newdf

In [ ]:
test_y

In [ ]:
sid

In [ ]:
tid

In [ ]:
tr.keys().to_list()

In [ ]:
tr.X2_abs_energy

In [ ]:
tr.X_abs_energy

In [ ]:
tr_y

In [ ]:
data = {'subject_id':[1,2,3,4], 'spcount':[6,7,8,10], 'feat1':[5.6,2.4,6.7,4.5], 'label':[1.0,5.0,6.0,9.0]} 

# Create DataFrame 
df = pd.DataFrame(data, columns=['subject_id', 'spcount', 'feat1', 'label'])

In [ ]:
df

In [ ]:
numerics = ['int16', 'int32', 'int64']

newdf = df.select_dtypes(include=numerics)
newdf

In [ ]:
df2 = df[df.columns.difference(['subject_id', 'spcount'])]

print(df[['subject_id','spcount']])
df2 = df2.mul(lambda_value)
print(df2)

result = pd.concat([df[['subject_id','spcount']], df2], axis=1, sort=False)
display(result)
# df3 = pd.merge(df[['subject_id','spcount']], df2)
# df3
#df.append(df.mul(0.5), ignore_index=True)

In [ ]:
df

In [ ]:
allo = pd.concat([df, result], ignore_index=True)

In [ ]:
allo